In [1]:
import keras

Using TensorFlow backend.


In [2]:
from keras.applications.resnet50 import ResNet50
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Input, GlobalAveragePooling2D, Dense
from keras.models import Model
from keras.optimizers import SGD
import numpy as np
import pickle as pkl
import datetime

# Load Cifar10 dataset

In [63]:
(x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data()

In [64]:
print('The training set has {0} samples with dimentions {1}.'.format(x_train.shape[0], x_train.shape[1:]))
print('The test set has {0} samples with dimentions {1}.'.format(x_test.shape[0], x_test.shape[1:]))

The training set has 50000 samples with dimentions (32, 32, 3).
The test set has 10000 samples with dimentions (32, 32, 3).


In [65]:
NUM_CLASSES = 10
IMG_ROWS, IMG_COLS = 32, 32
BATCH_SIZE = 512
EPOCHS = 20
VAL_SPLIT = .2
LEARNING_RATE = 1e-2
SGD_MOMENTUM = .9

# Preprocess data

In [66]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

In [67]:
x_train -= np.mean(x_train)
x_test -= np.mean(x_test)

In [68]:
x_train /= np.std(x_train)
x_test /= np.std(x_test)

In [69]:
y_train = keras.utils.to_categorical(y_train, NUM_CLASSES)
y_test = keras.utils.to_categorical(y_test, NUM_CLASSES)

In [70]:
val_split = int((1-VAL_SPLIT)*len(x_train))
x_train , x_val = x_train[:val_split], x_train[val_split:]
y_train, y_val = y_train[:val_split], y_train[val_split:]

# Define data augmentations

In [71]:
datagen = ImageDataGenerator(width_shift_range=.2, 
                             height_shift_range=.2,
                             zoom_range=.2,
                             horizontal_flip=True, 
                             vertical_flip=True)

# Build model

In [72]:
input_tensor = Input(shape=(IMG_ROWS, IMG_COLS, 3))

In [73]:
base_model = ResNet50(include_top=False, input_tensor=input_tensor)

In [75]:
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(NUM_CLASSES, activation='softmax')(x)

In [76]:
resnet50 = Model(inputs=base_model.input, outputs=predictions)

In [77]:
resnet50.compile(optimizer=SGD(lr=LEARNING_RATE, momentum=SGD_MOMENTUM), loss='categorical_crossentropy', metrics=['accuracy'])

# Train model

In [78]:
now = datetime.datetime.now()
tensorflow_log_dir = "Graph/" + now.strftime("%Y%m%d-%H%M%S") + "/"

In [79]:
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=tensorflow_log_dir, histogram_freq=0,  
          write_graph=True, write_images=False)

In [82]:
resnet50.fit_generator(datagen.flow(x_train, y_train, batch_size=BATCH_SIZE),
                    steps_per_epoch=len(x_train) / BATCH_SIZE, 
                       validation_data=ImageDataGenerator().flow(x_val, y_val, batch_size=BATCH_SIZE),
                       validation_steps=len(x_val) / BATCH_SIZE,
                       epochs=EPOCHS, callbacks=[tensorboard_callback], verbose=1)

KeyboardInterrupt: 

In [81]:
resnet50.evaluate(x_test, y_test, verbose=1)

10000/10000 [==============================] - 12s 1ms/step


[0.6781904963254929, 0.7864]